In [1]:
%%capture
!pip install transformers[torch]
!pip install kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
!pip install --upgrade -q wandb
#!python -m spacy download fr_core_news_sm  #download the french model for spacy when cleaning the data

In [2]:
%%capture
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
new_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/model2').to(device)

In [4]:
unlabelled_data = pd.read_csv('/kaggle/input/testdata/unlabelled_test_data.csv', index_col = 'id')

In [5]:
%%capture
new_tokenizer = AutoTokenizer.from_pretrained("camembert-base", do_lower_case=True)

In [6]:
## The code to remove the numbers. We didn't use it because it didn't improve the results. 

#import re
#from typing import Dict
#
#def remove_numbers(text):
#    # Use regular expression to remove numbers
#    text_without_numbers = re.sub(r'\d+', '', text)
#    return text_without_numbers

In [7]:
## The code to remove the proper names. We didn't use it because it didn't improve the results.

#import spacy
#
## Load the spaCy French language model
#nlp = spacy.load('fr_core_news_sm')
#
#def remove_proper_names(text):
#    doc = nlp(text)
#    # Remove entities (proper names)
#    text_without_entities = ' '.join([token.text if not token.ent_type_ else '' for token in doc])
#    return text_without_entities

In [8]:
## Processing of the data when we use the code to remove the numbers and the proper names. We didn't use it because it didn't improve the results.

#def process_data(row) -> Dict:
#    # Clean the text
#    text = row['sentence']
#    text = str(text)
#    text = remove_numbers(text)
#    text = remove_proper_names(text)
#
#    # Convert difficulty labels to integers
#    row['sentence'] = text
#
#    return row

In [10]:
## Processing of the data to clean the data with (numberts and proper names)

#processed_unlabelled_data = unlabelled_data.apply(process_data, axis=1)

In [11]:
def get_multiclass_predictions(texts):
    encodings = new_tokenizer(texts.tolist(), return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    encodings = {k: v.to(new_model.device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = new_model(**encodings)

    logits = outputs.logits

    softmax = torch.nn.Softmax(dim=-1)  # Adjust the dimension to -1
    probabilities = softmax(logits.squeeze().cpu())
    probabilities = probabilities.detach().numpy()

    labels = np.argmax(probabilities, axis=-1)

    label_mapping = {0: 'A1', 1: 'A2', 2: 'B1', 3: 'B2', 4: 'C1', 5: 'C2'}

    predicted_labels = [label_mapping[label.item()] for label in labels]

    return predicted_labels

In [13]:
#predicted_labels = get_multiclass_predictions(processed_unlabelled_data['sentence']) # use if data is processed
predicted_labels = get_multiclass_predictions(unlabelled_data['sentence'])

In [14]:
unlabelled_data['difficulty'] = predicted_labels

In [15]:
predictions = unlabelled_data[['difficulty']]

In [16]:
predictions.head()

,difficulty
id,
0,C2
1,A2
2,B1
3,A2
4,C2


In [17]:
predictions.to_csv('submission.csv')

In [18]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Camembert lr 5e-5"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
